# Теория игр: Практикум в Jupyter Notebook

Этот ноутбук предназначен для закрепления материала по **теории игр** на примерах и практических задачах. Мы разберём несколько матричных игр, научимся находить оптимальные стратегии в **чистом** и **смешанном** виде, а также познакомимся с библиотеками Python, которые упрощают расчёты и позволяют проводить небольшие симуляции.

## Краткое содержание
1. **Подготовка окружения** (библиотеки и функции).
2. **Пример 1**: Игра 2×2 с седловой точкой.
3. **Пример 2**: Игра 2×2 без седловой точки (поиск смешанных стратегий).
4. **Пример 3**: "Камень-ножницы-бумага" (3×3). Симуляция.
5. (Опционально) **Линейное программирование** для решения матричной игры.
7. Дополнительные практические задания с решениями (3 уровня сложности).

Каждый раздел содержит краткую теорию, кодовые примеры, а также задания для самостоятельной работы.

> **Совет**: чтобы по-настоящему разобраться в материале, обязательно экспериментируйте с кодом, меняйте параметры матриц, пробуйте разные вероятности стратегий и смотрите на результаты.

## 1. Подготовка окружения

В этом разделе:
1. Подключим основные библиотеки.
2. Определим вспомогательную функцию `expected_payoff`.

**Используемые библиотеки**:
- **NumPy**: работа с матрицами и векторами.
- **Nashpy**: поиск равновесий Нэша в матричных играх.

### Функция `expected_payoff`
Она считает математическое ожидание выигрыша первого игрока при заданных **смешанных** стратегиях:
- `A`: платежная матрица (выигрыш первого игрока);
- `P`: вероятности (распределение) первого игрока;
- `Q`: вероятности второго игрока.
Возвращает число (скаляр) — ожидаемый выигрыш первого игрока.

In [12]:
import numpy as np
import nashpy as nash

def expected_payoff(A, P, Q):
    """
    Возвращает ожидаемый выигрыш первого игрока.
    A : np.ndarray (матрица выигрышей первого игрока)
    P : список или массив вероятностей для первого игрока
    Q : список или массив вероятностей для второго игрока
    """
    # Преобразуем P, Q в numpy-массивы, чтобы удобно делать матричные операции
    P = np.array(P)
    Q = np.array(Q)
    # Математическое ожидание: P * A * Q^T
    return P.dot(A).dot(Q.T)

## 2. Пример 1: Игра 2×2 с седловой точкой

**Теория (кратко)**:
- Если в матрице $A$ существует элемент, который одновременно:
  1) **максимален в своей строке**;
  2) **минимален в своём столбце**,
  то этот элемент называется **седловой точкой**. Такие пары стратегий в чистом виде дают равновесие (никто из игроков не имеет стимула отклоняться).

**Матрица**:
$$
A = \begin{pmatrix}
3 & 2 \\
4 & 1
\end{pmatrix}
$$
Где $A_{ij}$ — выигрыш первого игрока.

### 2.1. Код для поиска седловой точки
Выполним поиск автоматически, проверяя какие элементы являются макс. в строке и мин. в столбце.

> Обратите внимание, что если **такого элемента нет**, игра не имеет равновесия в чистых стратегиях и игрокам придётся использовать **смешанные** стратегии.

In [6]:
# Пример платежной матрицы 2x2
A = np.array([
    [3, 2],
    [4, 1]
])

# Максимум в каждой строке
row_max = A.max(axis=1)
# Минимум в каждом столбце
col_min = A.min(axis=0)

print("Матрица A:\n", A)
print("Максимумы строк:", row_max)
print("Минимумы столбцов:", col_min)

sedlovye_tochki = []
for i in range(A.shape[0]):
    for j in range(A.shape[1]):
        # Проверяем условие "макс в строке, мин в столбце"
        if A[i, j] == row_max[i] and A[i, j] == col_min[j]:
            sedlovye_tochki.append((i, j))

if sedlovye_tochki:
    print("Седловая точка (точки):", sedlovye_tochki)
    for (i,j) in sedlovye_tochki:
        print(f"Седловая точка = A[{i},{j}] =", A[i,j])
else:
    print("Седловая точка не найдена.")

Матрица A:
 [[3 2]
 [4 1]]
Максимумы строк: [3 4]
Минимумы столбцов: [3 1]
Седловая точка (точки): [(0, 0)]
Седловая точка = A[0,0] = 3


### 2.2. Задание для самостоятельной работы (Задание 1)

**Задание**: Измените значения в матрице `A` и проверьте, будет ли седловая точка. Попробуйте:
1. `A = [[2, 2],[2, 2]]` (все элементы одинаковые).
2. `A = [[0, 4],[1, 3]]`.
3. Любую другую 2×2 матрицу, которую захотите.

После запуска кода посмотрите, найдена ли седловая точка, и каков её выигрыш.

*Подумайте*, что означает ситуация без седловой точки: почему игроки не могут просто выбрать чистые стратегии и быть уверенными в «лучшем» результате?

## 3. Пример 2: Игра 2×2 без седловой точки

**Матрица**:
$$
A = \begin{pmatrix}
1 & 3 \\
2 & 0
\end{pmatrix}
$$
Теория игр подсказывает, что для этой матрицы **седловой точки нет**. Следовательно, игроки должны использовать **смешанные** стратегии, чтобы достичь равновесия.

### 3.1. Использование библиотеки Nashpy
Создадим объект `Game(A)` и воспользуемся методом `support_enumeration()` для поиска равновесий Нэша. Если равновесие существует, он нам его покажет (с точки зрения распределений $p$ и $q$).

In [13]:
A = np.array([
    [1, 3],
    [2, 0]
])
print("Матрица A:\n", A)

# Создаём игру на базе матрицы A
game_2x2 = nash.Game(A)
equilibria = list(game_2x2.support_enumeration())

print("Найдены равновесия (p*, q*):")
for eq in equilibria:
    print(eq)

# Обычно вернётся один кортеж (p_star, q_star)
p_star, q_star = equilibria[0]
print("\nОптимальная стратегия первого игрока:", p_star)
print("Оптимальная стратегия второго игрока:", q_star)

# Вычислим значение игры - ожидаемый выигрыш первого игрока
V = expected_payoff(A, p_star, q_star)
print("Значение игры (выигрыш первого игрока):", V)

Матрица A:
 [[1 3]
 [2 0]]
Найдены равновесия (p*, q*):
(array([0.5, 0.5]), array([0.75, 0.25]))

Оптимальная стратегия первого игрока: [0.5 0.5]
Оптимальная стратегия второго игрока: [0.75 0.25]
Значение игры (выигрыш первого игрока): 1.5


### 3.2. Задание для самостоятельной работы (Задание 2)
1. **Аналитическое уравнение**. Попробуйте вручную (на бумаге) вывести уравнение безразличия для первого и второго игрока. Получите те же самые $p^*$ и $q^*$. Сравните с результатом `support_enumeration()`.
2. **Измените матрицу** `A` на другую 2×2 (где нет седловой точки). Убедитесь, что Nashpy находит другие распределения $p, q$.
3. **Попробуйте** оценить `expected_payoff(A, p, q)` для нескольких комбинаций $p, q$ (например, 0.0, 0.25, 0.5, 1.0) и посмотрите, как это влияет на выигрыш.


## 4. Пример 3: "Камень-ножницы-бумага" (3×3). Симуляция

### 4.1. Постановка задачи
Классическая игра: **камень, ножницы, бумага**. Для первого игрока зададим платежную матрицу:
$$
A = \begin{pmatrix}
 0 & -1 & 1  \\
 1 &  0 & -1 \\
 -1 & 1 &  0
\end{pmatrix}
$$

Смысл:
- Камень (0) бьёт ножницы (2),
- Бумага (1) бьёт камень (0),
- Ножницы (2) бьют бумагу (1).

В теории хорошо известно, что равновесие здесь — **равномерное** $(1/3, 1/3, 1/3)$. Но мы проверим это с помощью `nashpy` и короткой симуляции.

In [14]:
# Классическая матрица (RPS)
A = np.array([
    [0, -1,  1],
    [1,  0, -1],
    [-1, 1,  0]
])

game_rps = nash.Game(A)
eqs_rps = list(game_rps.support_enumeration())
print("Найдены равновесия Нэша (первый игрок, второй игрок):")
for eq in eqs_rps:
    print(eq)

Найдены равновесия Нэша (первый игрок, второй игрок):
(array([0.33333333, 0.33333333, 0.33333333]), array([0.33333333, 0.33333333, 0.33333333]))


### 4.3. Симуляция случайных партий

Чтобы увидеть, как стратегии работают "в реальном бою", устроим несколько раундов. Например:
- Первый игрок играет строго $(1/3, 1/3, 1/3)$.
- Второй игрок предпочитает **камень** на 90%, а ножницы и бумагу лишь по 5%.

Посмотрим, какой **средний** выигрыш получится у первого игрока за N=100000 раундов.

In [15]:
import random

def single_round_payoff(A, strategy1, strategy2):
    """
    Одна партия игры: первый игрок выбирает i по strategy1,
    второй – j по strategy2. Возвращаем A[i, j].
    """
    # i – индекс строки (стратегия 1-го игрока)
    i = np.random.choice(len(strategy1), p=strategy1)
    # j – индекс столбца (стратегия 2-го игрока)
    j = np.random.choice(len(strategy2), p=strategy2)
    return A[i, j]

# Матрица RPS та же
A_rps = np.array([
    [0, -1,  1],
    [1,  0, -1],
    [-1, 1,  0]
])

strategy_1 = [1/3, 1/3, 1/3]  # (камень, бумага, ножницы) равномерно
strategy_2 = [0.9, 0.05, 0.05] # преимущественно камень

N = 100000
total_payoff = 0
for _ in range(N):
    total_payoff += single_round_payoff(A_rps, strategy_1, strategy_2)

avg_payoff = total_payoff / N
print("При стратегии первого (1/3,1/3,1/3) и второго (0.9,0.05,0.05):")
print("Средний выигрыш первого игрока:", avg_payoff)

При стратегии первого (1/3,1/3,1/3) и второго (0.9,0.05,0.05):
Средний выигрыш первого игрока: -0.00153


### 4.4. Задание для самостоятельной работы (Задание 3)
1. **Измените** стратегию второго игрока (например, 50% камень, 30% ножницы, 20% бумага) и запустите симуляцию, чтобы узнать средний результат.
2. **Попробуйте** заставить обоих игроков играть равномерно (1/3, 1/3, 1/3). Проверьте, что средний выигрыш первого игрока близок к 0 (увеличьте `N`, чтобы уменьшить флуктуации).
3. **(Дополнительно)** Попробуйте немного менять стратегию первого игрока (увеличить/уменьшить долю «бумаги»), чтобы увидеть, меняется ли средний выигрыш. Должен ли он по идее становиться больше 0?

## 5. (Опционально) Линейное программирование в SciPy

Чтобы показать альтернативный общий метод решения (особенно для "минимакс" задач), можно сводить поиск оптимальной стратегии первого игрока к **задаче линейного программирования**. Ниже — пример кода, как это реализовать для 2×2.

> **Примечание**: Этот блок можно пропустить, если фокус урока — только на `nashpy` и базовых симуляциях, без погружения в LP.

In [11]:
from scipy.optimize import linprog

# Пример: A = [[1,3],[2,0]] (из Примера 2)
A_lp = np.array([
    [1, 3],
    [2, 0]
])
m, n = A_lp.shape

# Переменные: p_1, p_2, ..., p_m, v
# Цель: макс v <=> min -v
# Вектор c: у нас m+1 переменных (p_i и v)
c = np.zeros(m + 1)
c[-1] = -1  # т.к. минимизируем -v

# Ограничение: sum(p_i) = 1
A_eq = np.ones((1, m+1))
A_eq[0, -1] = 0  # v не участвует в сумме p_i
b_eq = np.array([1])

# Неравенства: sum_i(p_i*A[i,j]) >= v, для каждого j
# Перепишем: - sum_i(p_i*A[i,j]) + v <= 0
A_ineq = []
b_ineq = []

for j in range(n):
    row = np.zeros(m + 1)
    for i in range(m):
        row[i] = -A_lp[i,j]
    row[-1] = 1  # коэффициент при v
    A_ineq.append(row)
    b_ineq.append(0)

A_ineq = np.array(A_ineq)
b_ineq = np.array(b_ineq)

# p_i >= 0, v может быть любым (но фактически ограничивается условиями)
bounds = [(0, None)]*m + [(None, None)]

res = linprog(c, A_ineq, b_ineq, A_eq, b_eq, bounds=bounds, method='highs')
if res.success:
    p = res.x[:-1]
    v = res.x[-1]
    print("Оптимальная стратегия первого игрока:", p / p.sum())
    print("Значение игры:", v)
else:
    print("LP не нашёл решения.")

Оптимальная стратегия первого игрока: [0.5 0.5]
Значение игры: 1.5


## 6. Практические задания с решениями

Ниже приведён набор заданий **трёх уровней сложности** (лёгкий, средний, сложный). В каждом уровне по три пункта. После формулировки приведены **решения** для самопроверки — **каждое решение находится в отдельной кодовой ячейке**.

> **Важно**: сначала постарайтесь выполнить задания самостоятельно, а только потом смотрите решения.

---
### 6.1. Уровень «Лёгкий»

**(A1)** Возьмите матрицу из Примера 1 (2×2) и вручную (или с помощью кода) проверьте, почему в этой игре есть седловая точка.

**(A2)** Измените матрицу Примера 1 так, чтобы не было седловой точки. Убедитесь в этом программно. Подсказка: поменяйте 1–2 элемента, чтобы нарушилось условие "макс в строке = мин в столбце".

**(A3)** Используя функцию `expected_payoff`, посчитайте ожидаемый выигрыш первого игрока в Примере 2 (матрица `[[1,3],[2,0]]`), если первый игрок всегда играет первую строку (100%), а второй — первый столбец (100%).

#### Решения (Лёгкий)

**(A1) Решение**

**(A2) Решение**

**(A3) Решение**

---
### 6.2. Уровень «Средний»

**(B1)** В Примере 2 (`[[1,3],[2,0]]`) найдите вручную (на бумаге) оптимальную смешанную стратегию первого игрока.

**(B2)** Измените код для игры "Камень-ножницы-бумага" (3×3), чтобы второй игрок играл 70% камень, 30% ножницы (0% бумага). Каков будет средний выигрыш первого?

**(B3)** Вспомните ещё одну игру 2×2 (например, про "высокую" или "низкую" цену). Составьте матрицу $2\times2$. Определите через `nashpy`, есть ли седловая точка.

#### Решения (Средний)

**(B1) Решение**

**(B2) Решение**

**(B3) Решение**

---
### 6.3. Уровень «Сложный»

**(C1)** Составьте **3×3** игру (с нулевой суммой), где нет равновесия в чистых стратегиях. Найдите равновесие в смешанных стратегиях через `nashpy`. Сравните ожидаемый выигрыш, если один из игроков отклонится.

**(C2)** Напишите (или допишите) небольшой цикл "обучения" в симуляции, где оба игрока корректируют свои вероятности шаг за шагом.

**(C3)** Примените подход **линейного программирования** для 3×3 или используйте `scipy.optimize.linprog` для $p_1, p_2, p_3$. Сравните результаты с `support_enumeration()`.

#### Решения (Сложный)

**(C1) Решение**

**(C2) Решение**

**(C3) Решение**